<a href="https://colab.research.google.com/github/thePegasusai/CloudPhone/blob/main/Color(Draft).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Install TensorFlow if not already installed
!pip install tensorflow

# Import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set a random seed for reproducibility
tf.random.set_seed(42)

In [9]:
# Load the Excel file
df = pd.read_excel('/Colors.xlsx')  # Make sure 'Colors.xlsx' is uploaded to your Colab session

# Calculate how many complete 4x4 images can be created
num_complete_images = (rgb_values.shape[0] // 48) * 48

# Reshape only the complete images
rgb_values_reshaped = rgb_values[:num_complete_images].reshape(-1, 4, 4, 3)

# Handle leftover data (if any)
leftover_data = rgb_values[num_complete_images:]  # This data will be discarded

# Now use `rgb_values_reshaped` for your model training

In [10]:
# Define the encoder
encoder = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(4, 4, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),  # Latent space representation
])

# Define the decoder
decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(128,)),
    tf.keras.layers.Reshape((1, 1, 128)),
    tf.keras.layers.Conv2DTranspose(64, (3, 3), activation='relu'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2DTranspose(32, (3, 3), activation='relu'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'),
])

ValueError: One of the dimensions in the output is <= 0 due to downsampling in conv2d_3. Consider increasing the input size. Received input shape [None, 1, 1, 32] which would produce output shape with a zero or negative value in a dimension.

In [ ]:
# Define the forward diffusion process (adding noise)
def add_noise(x, timesteps):
    noise = tf.random.normal(shape=x.shape)
    return x + (1 - timesteps) * noise  # Add more noise at earlier timesteps

# Define the reverse diffusion process (removing noise)
def remove_noise(x, timesteps):
    return decoder(x)  # Decoder tries to remove noise

In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError()

# Training loop with batching
epochs = 100
batch_size = 32
num_batches = (rgb_values.shape[0] + batch_size - 1) // batch_size

for epoch in range(epochs):
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = min((batch + 1) * batch_size, rgb_values.shape[0])
        batch_data = rgb_values[start_idx:end_idx]

        with tf.GradientTape() as tape:
            noisy_data = add_noise(batch_data, timesteps)
            denoised_data = remove_noise(noisy_data, timesteps)
            loss = loss_fn(denoised_data, batch_data)

        grads = tape.gradient(loss, decoder.trainable_variables)
        optimizer.apply_gradients(zip(grads, decoder.trainable_variables))

    print(f'Epoch {epoch+1}: Loss = {loss.numpy()}')

In [ ]:
# Generate new colors
num_colors = 10
noise = tf.random.normal(shape=(num_colors, 4, 4, 3))
generated_colors = remove_noise(noise, timesteps).numpy() * 255.0

# Visualize the generated colors
plt.figure(figsize=(8, 4))
for i in range(num_colors):
    plt.subplot(2, 5, i + 1)
    plt.imshow(generated_colors[i].reshape(4, 4, 3), interpolation='nearest')
    plt.axis('off')
plt.show()

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Data Loading & Preprocessing
df = pd.read_excel('Colors.xlsx')
rgb_values = df[['Red', 'Green', 'Blue']].values
rgb_values = rgb_values / 255.0  # Normalize to range [0, 1]

# 2. Model Architecture (Encoder & Decoder)
encoder = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(1, 1, 3)),  # Assuming 1x1 image for RGB
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),  # Latent space representation
])

decoder = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(128,)),
    tf.keras.layers.Reshape((1, 1, 128)),  # Reshape to match encoder output
    tf.keras.layers.Conv2DTranspose(64, (3, 3), activation='relu'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2DTranspose(32, (3, 3), activation='relu'),
    tf.keras.layers.UpSampling2D((2, 2)),
    tf.keras.layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same'),  # Output 3 channels (RGB)
])

# 3. Diffusion Process (Simplified)
def add_noise(x, timesteps):
    noise = tf.random.normal(shape=x.shape)
    return x + (1 - timesteps) * noise

def remove_noise(x, timesteps):
    return decoder(x)

# 4. Model Compilation and Training
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.MeanSquaredError()

# Training loop with batching
epochs = 100  # Adjust as needed
batch_size = 32  # Adjust as needed
num_batches = (rgb_values.shape[0] + batch_size - 1) // batch_size

for epoch in range(epochs):
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = min((batch + 1) * batch_size, rgb_values.shape[0])
        batch_data = rgb_values[start_idx:end_idx]

        with tf.GradientTape() as tape:
            noisy_data = add_noise(batch_data, timesteps)
            denoised_data = remove_noise(noisy_data, timesteps)
            loss = loss_fn(denoised_data, batch_data)

        grads = tape.gradient(loss, decoder.trainable_variables)
        optimizer.apply_gradients(zip(grads, decoder.trainable_variables))

    print(f'Epoch {epoch+1}: Loss = {loss.numpy()}')

# 5. Generating New Colors
num_colors = 10  # Adjust as needed
noise = tf.random.normal(shape=(num_colors, 1, 1, 3))
generated_colors = remove_noise(noise, timesteps).numpy() * 255.0  # De-normalize

# 6. Visualizing the Colors
plt.figure(figsize=(8, 4))
for i in range(num_colors):
    plt.subplot(2, 5, i + 1)
    plt.imshow(generated_colors[i].reshape(1, 1, 3), interpolation='nearest')
    plt.axis('off')
plt.show()